In [1]:
import json
import requests
import utils
import threading
from datetime import datetime as dt
from playsound import playsound
from configs import API_KEY,BASE_URL,AMOUNT,MIN_PERCENT_REQUIRED
from biscoint import get_rate_limit,get_balance,get_offer,confirm_offer
import time

In [2]:
#Inicial configs
rate_limit = get_rate_limit()
sleep_time_offers = ((rate_limit["windowMs"] / rate_limit["maxRequests"]) / 1000)*2
initial_balance = get_balance()
last_balance = initial_balance

In [7]:
%%time
buy = get_offer('buy','0.0001',False)
sell = get_offer('sell','0.0001',False)
print(sell)

{'offerId': 'cFWvDLznKYWg7FfYs', 'base': 'BTC', 'quote': 'BRL', 'op': 'sell', 'isQuote': False, 'baseAmount': '0.00010000', 'quoteAmount': '32.29', 'efPrice': '322900.00', 'createdAt': '2021-04-19T01:32:38.470Z', 'expiresAt': '2021-04-19T01:32:53.470Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}
Wall time: 2.03 s


In [37]:
def async_offer(op:str,amount:str,isQuote:bool):
    try:
        response = get_offer(op,amount,isQuote)
        request_orders[op] = response
    except Exception as e:
        print("Erro ao rodar thread",e)

In [43]:
while True:
    start_time = dt.now()
    try:
        request_orders = {'buy':None,'sell':None}

        # thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
        # thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
        # thread_buy.start()
        # time.sleep(0.1)
        # thread_sell.start()
        # thread_buy.join()
        # thread_sell.join()
        request_orders['buy'] = get_offer('buy',AMOUNT,False)
        request_orders['sell'] = get_offer('sell',AMOUNT,False)

        buy_price = request_orders['buy']['efPrice']
        sell_price = request_orders['sell']['efPrice']
        percent = utils.percent(buy_price, sell_price)

        if percent >= MIN_PERCENT_REQUIRED:
            print(f"Arbitrage oportunity: buy:{buy_price}   sell:{sell_price} ->  {percent}%")
            playsound('beep.wav')

        print(percent)

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        print(f"Took {seconds_elapsed} seconds")
    
        time.sleep(sleep_time_offers-seconds_elapsed)
    except Exception as e:
        print(e)



-0.615
Took 1.790898 seconds
-0.615
Took 1.361486 seconds
-0.308
Took 2.148119 seconds
-0.613
Took 0.990161 seconds
-0.615
Took 0.798309 seconds


KeyboardInterrupt: 

In [22]:
#confirm_offer('EAWBrEhsrAtbe8BXE')
buy = get_offer('buy','0.00018',False)
print(buy)
#confirm_offer(buy['offerId'])

{'offerId': 't42LRiSJ8Ki7jjxhu', 'base': 'BTC', 'quote': 'BRL', 'op': 'buy', 'isQuote': False, 'baseAmount': '0.00018000', 'quoteAmount': '58.62', 'efPrice': '325666.67', 'createdAt': '2021-04-19T10:37:19.983Z', 'expiresAt': '2021-04-19T10:37:34.983Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}


In [23]:
sell = get_offer('sell','0.00018',False)
print(sell)

{'offerId': 'vn3wEpkeHAR4Ti7xj', 'base': 'BTC', 'quote': 'BRL', 'op': 'sell', 'isQuote': False, 'baseAmount': '0.00018000', 'quoteAmount': '57.70', 'efPrice': '320555.56', 'createdAt': '2021-04-19T10:37:57.072Z', 'expiresAt': '2021-04-19T10:38:12.072Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}
